In [208]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import folium
from tqdm import tqdm
import os
import time
from collections import deque
import json
import requests

In [209]:
class StationEdgeScraper :
    def __init__(self) :
        options = webdriver.ChromeOptions()
        # options.add_argument("--headless")  # Run browser in headless mode
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        
        self.edges = dict()
        
    def get_edges(self, start_url) :
        self.driver.get(start_url)
        self.driver.implicitly_wait(5)
        
        name = self.__get_current_station_name()
        q = deque([(name, start_url)])
        while q :
            cur_station, cur_url = q.popleft()
            if cur_station not in self.edges :
                self.edges[cur_station] = []
            if self.driver.current_url != cur_url :
                self.driver.get(cur_url)
                self.driver.implicitly_wait(5)
            
            nxt_urls = self.__get_next_url()
            for nxt_station, nxt_url in nxt_urls :
                if nxt_station in self.edges[cur_station] :
                    continue
                self.edges[cur_station].append(nxt_station)
                q.append([nxt_station, nxt_url])
                
        self.driver.quit()

    def __get_current_station_name(self) :
        return self.driver.find_element(By.CLASS_NAME, "station_main_area").find_element(By.CLASS_NAME, "station").text + '역'
    
    def __get_next_url(self) :
        ret = []
        button = self.driver.find_element(By.CLASS_NAME, "subway_content_wrap").find_element(By.CLASS_NAME, "btn_next")
        next_station_count = len(button.find_element(By.CLASS_NAME, "station").find_elements(By.TAG_NAME, "span"))-1
        
        if next_station_count == 0 :
            return ret
        elif next_station_count == 1 :
            ActionChains(self.driver).click(button).perform()
            name = self.__get_current_station_name()
            ret.append((name, self.driver.current_url))
        
        else :
            for i in range(next_station_count) :
                button = self.driver.find_element(By.CLASS_NAME, "subway_content_wrap").find_element(By.CLASS_NAME, "btn_next")
                ActionChains(self.driver).click(button).perform()
                self.driver.implicitly_wait(5)
                
                buttons = self.driver.find_element(By.CLASS_NAME, "list_prev_next_station").find_elements(By.TAG_NAME, "button")
                ActionChains(self.driver).click(buttons[i]).perform()
                name = self.__get_current_station_name()
                ret.append((name, self.driver.current_url))
                self.driver.back()
                self.driver.implicitly_wait(5)
            
        return ret
    

api_key = ""

def get_lat_lon(addr) :
    
    headers = {"Authorization": f'KakaoAK {api_key}'} #REST API 키(유효한 키)
    url = f'https://dapi.kakao.com/v2/local/search/keyword.json?query={addr}'
    result = requests.get(url, headers = headers).json()
    
    lat = float(result['documents'][0]['y'])
    lon = float(result['documents'][0]['x'])
    
    return lat, lon

In [210]:
point_scraper = PointScraper()
station_edge_scraper = StationEdgeScraper()

start_station_name_1st = "별내역"
start_station_url_1st = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&ssc=tab.nx.all&query=%EB%B3%84%EB%82%B4%EC%97%AD&oquery=%EC%9E%A5%EC%95%94%EC%97%AD&tqi=iw9%2B7dqVOswssjKceXGssssssSo-441899"


In [211]:
station_edge_scraper.get_edges(start_station_url_1st)
edges = station_edge_scraper.edges

In [212]:
points = dict()
for station in edges.keys() :
    points[station] = get_lat_lon(station)
    

In [213]:
def get_middle(points) :
    sum_a = 0
    sum_b = 0
    for a,b in points.values() :
        sum_a += a
        sum_b += b
        
    return sum_a/len(points), sum_b/len(points)

map = folium.Map(location=get_middle(points), zoom_start=11)

q = deque([start_station_name_1st])
visited = set()

while q :
    cur_station = q.popleft()
    if cur_station in visited :
        continue
    
    folium.Marker(points[cur_station], popup=cur_station).add_to(map)
    visited.add(cur_station)
    
    for nxt_station in edges[cur_station] :
        q.append(nxt_station)
        folium.PolyLine([points[cur_station], points[nxt_station]], color="blue", weight=2.5, opacity=1).add_to(map)
        
map.save("station_edge_visualization.html")

In [214]:
map